# 3.6 Softmax regression from scratch

In [3]:
from IPython import display
from mxnet import autograd, gluon, np, npx
from d2l import mxnet as d2l

npx.set_np()

In [4]:
batch_size=256
train_iter, test_iter=d2l.load_data_fashion_mnist(batch_size)

[10:48:47] /home/mertins/Desenvolvimento/C/Terceiros/mxnet/src/storage/storage.cc:202: Using Pooled (Naive) StorageManager for CPU


[ INFO:0] global /builddir/build/BUILD/opencv-4.5.3/modules/core/src/parallel/registry_parallel.impl.hpp (96) ParallelBackendRegistry core(parallel): Enabled backends(2, sorted by priority): TBB(1000); OPENMP(990)
[ INFO:0] global /builddir/build/BUILD/opencv-4.5.3/modules/core/include/opencv2/core/parallel/backend/parallel_for.tbb.hpp (54) ParallelForBackend Initializing TBB parallel backend: TBB_INTERFACE_VERSION=11103
[ INFO:0] global /builddir/build/BUILD/opencv-4.5.3/modules/core/src/parallel/parallel.cpp (77) createParallelForAPI core(parallel): using backend: TBB (priority=1000)
[ INFO:0] global /builddir/build/BUILD/opencv-4.5.3/modules/core/src/parallel/registry_parallel.impl.hpp (96) ParallelBackendRegistry core(parallel): Enabled backends(2, sorted by priority): TBB(1000); OPENMP(990)
[ INFO:0] global /builddir/build/BUILD/opencv-4.5.3/modules/core/include/opencv2/core/parallel/backend/parallel_for.tbb.hpp (54) ParallelForBackend Initializing TBB parallel backend: TBB_INTERF

In [5]:
num_inputs=784
num_outputs=10

w=np.random.normal(0,0.01,(num_inputs,num_outputs))
b=np.zeros(num_outputs)
w.attach_grad()
b.attach_grad()

In [6]:
X=np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])
X.shape,X.sum(0,keepdims=True),X.sum(1,keepdims=True)

((2, 3),
 array([[5., 7., 9.]]),
 array([[ 6.],
        [15.]]))

In [7]:
def softmax(X):
    X_exp=np.exp(X)
    partition = X_exp.sum(1,keepdims=True)
    return X_exp/partition

In [8]:
X = np.random.normal(0,1,(2,5))
X_prob = softmax(X)
X,X_prob, X_prob.sum(1)

(array([[ 0.5030464 , -0.7089397 , -0.72034746,  0.79505837,  0.9336002 ],
        [ 1.0187219 , -1.97108   , -0.15738758, -0.907509  , -1.5379014 ]]),
 array([[0.22376052, 0.06659239, 0.06583703, 0.29964197, 0.3441681 ],
        [0.63209665, 0.03179282, 0.194987  , 0.09209415, 0.04902935]]),
 array([1., 1.]))

In [9]:
def net(X):
    return softmax(np.dot(X.reshape((-1,W.shape[0])),W)+b)

# Loss function

In [10]:
y=np.array([0,2])
y_hat=np.array([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],y]

array([0.1, 0.5])

In [11]:
def cross_entropy(y_hat,y):
    return -np.log(y_hat[range(len(y_hat)),y])

cross_entropy(y_hat,y)

array([2.3025851, 0.6931472])

#### 3.6.5 Classification Accuracy

In [12]:
def accuracy(y_hat,y):
    if (len(y_hat.shape)>1 and y_hat.shape[1]>1):
        y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.astype(y.dtype)==y
    return float(cmp.astype(y.dtype).sum())

In [13]:
accuracy(y_hat, y)/len(y)

0.5

In [14]:
class Accumulatro:
    def __init__(self,n):
        self.data=[0,0]*n
    
    def add(self,*args):
        self.data=[a+float(b) for a,b in zip(self.data,args)]
    def reset(self):
        self.data=[0,0]*len(self.data)
    def __getitem__(self,idx):
        return self.data[idx]

In [15]:
def evaluate_accurary(net,data_iter):
    metric=Accumulator(2)
    for X,y in data_iter:
        metric.add(accuracy(net(X),y),d2l.size(y))
    return metric[0]/metric[1]